In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pickle
from scipy.special import expit
import json

import numpy as np
from sklearn.utils import shuffle
import pandas as pd
import seaborn as sns

from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.decomposition import PCA

import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt
import pickle
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler, StandardScaler
import numpy as np
from sklearn.decomposition import PCA

# Synthetic Data for Selection Bias

In [ ]:
def filter_data(data, y, c, n, p_y, p_c, dataframe=False):
    '''
        new: filter data to have total n data points where it has event rate (in uncensored) as p_y and censoring rate in total as p_c
    '''
    df = pd.DataFrame(data)
    df['y'] = y
    df['c']= c

    num_c = int(n * p_c)
    num_y = int((n-num_c) * p_y)
    # print(num_c, num_y)

    df_c = df.loc[df['c']==1].iloc[:num_c,:]

    df_y1 = df.loc[(df['c'] == 0) & (df['y'] == 1)].iloc[:num_y,:]
    df_y0 = df.loc[(df['c'] == 0) & (df['y'] == 0)].iloc[:(n-num_y-num_c),:]

    results = pd.concat([df_c, df_y1, df_y0], axis=0, ignore_index=True)
    results = shuffle(results)

    df = results.drop(columns=['y', 'c'])

    if dataframe:
        y = results['y']
        c = results['c']
        X = df
    else:
        y = results['y'].values
        c = results['c'].values
        X = df.values

    return X, y, c

def check_least_populated(X, y, c):
    if len(X[(y==0) & (c==0)])==1:
        con = ((y==0) & (c==0))
        X = X[~con].reset_index(drop=True)
        y = y[~con].reset_index(drop=True)
        c = c[~con].reset_index(drop=True)
    elif len(X[(y==0) & (c==1)])==1:
        con = ((y==0) & (c==1))
        X = X[~con].reset_index(drop=True)
        y = y[~con].reset_index(drop=True)
        c = c[~con].reset_index(drop=True)
    elif len(X[(y==1) & (c==0)])==1:
        con = ((y==1) & (c==0))
        X = X[~con].reset_index(drop=True)
        y = y[~con].reset_index(drop=True)
        c = c[~con].reset_index(drop=True)
    elif len(X[(y==1) & (c==1)])==1:
        con = ((y==1) & (c==1))
        X = X[~con].reset_index(drop=True)
        y = y[~con].reset_index(drop=True)
        c = c[~con].reset_index(drop=True)

    return X, y, c

def generate_classification_data(m, c, num_samples, num_features, nonlinear=True, nlband=10, flip_y=0.01):
    # Generate features data
    features = np.random.uniform(-10, 10, size=(num_samples, num_features))

    # generate labels
    linear_output1 = np.dot(features**1, m) + c
    labels = (linear_output1>0).astype(int)

    # Randomly replace labels
    # print('before:', labels.sum())
    if flip_y >= 0.0:
        flip_mask = np.random.uniform(size=labels.shape[0]) < flip_y
        labels[flip_mask] = np.random.randint(2, size=flip_mask.sum())
    # print('after:', labels.sum())
    
    # generate censoring
    m[1] *= 10
    c = -5
    linear_output2 = np.dot(features**1, m) + c
    sensoring = ((linear_output1>0) ^ (linear_output2>0)).astype(int)

    return features, labels, sensoring

def get_dataset_combinations(risk_rate=[.1, .05], censoring_rate=[.1, .05], n=[2000, 1000], test_size=0.2):
  '''
    This will generate data for different combinations of risk, censoring and dataset size.
        n - is total number of data points
        censoring_rate - rate of censoring and calculated on n
        risk_rate - event rate and calculated on uncensored data (i.e., after removing censored units)
        test_size - test and validation split fraction

    '''
  data_dict = {}

  np.random.seed(42)  # Set seed for reproducibility
  num_samples = 300000
  num_features = 25

  m = np.random.uniform(-1, 1, size=num_features)  # Slope/coefficients for features
  c = np.random.uniform(-5, 5)  # Intercept

  data, labels, sensoring = generate_classification_data(m, c, num_samples, num_features, nonlinear=True, nlband=10)
  labels = labels.astype(float)
  print('labels.shape:', labels.shape, 'labels[sensoring==0].sum():', labels[sensoring==0].sum(), 'sensoring.sum():', sensoring.sum())

  risk_rate = sorted(risk_rate, reverse=True)
  censoring_rate = sorted(censoring_rate, reverse=True)
  n = sorted(n, reverse=True)
  
  for rr in risk_rate:
      for cc in censoring_rate:
          print('-----------------------------------------')
          X, y, c = filter_data(data.copy(), labels.copy(), sensoring.copy(), n=n[0], p_y=rr, p_c=cc)
          X_train, X_test, y_train, y_test, c_train, c_test = train_test_split(X.copy(), y.copy(), c.copy(), test_size=test_size, random_state=0, stratify= y*2 + c)
          X_train, X_val, y_train, y_val, c_train, c_val = train_test_split(X_train, y_train, c_train, test_size=test_size, random_state=0, 
                                                                            stratify= y_train*2 + c_train)
          
          data_dict['X_train'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = X_train
          data_dict['y_train'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = y_train
          data_dict['c_train'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = c_train
          data_dict['X_val'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = X_val
          data_dict['y_val'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = y_val
          data_dict['c_val'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = c_val
          data_dict['X_test'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = X_test
          data_dict['y_test'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = y_test
          data_dict['c_test'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = c_test

          for i in range(1, len(n)):
              # we know total size of dataset whihc has to be split into train/val/test. Further
              # desired dataset to be subset of bigger dataset
              # check least populated and remove if there is only one sample to avoid splitting error.
            #   X_train, y_train, c_train = check_least_populated(X_train, y_train, c_train)
            #   X_val, y_val, c_val = check_least_populated(X_val, y_val, c_val)
              _, X_train, _, y_train, _, c_train = train_test_split(X_train, y_train, c_train, test_size=int(n[i]*(1-test_size)*(1-test_size)), 
                                                                    random_state=42, stratify=y_train*2 + c_train)
              _, X_val, _, y_val, _, c_val = train_test_split(X_val, y_val, c_val, test_size=int(n[i]*(1-test_size)*test_size), random_state=42, 
                                                              stratify=y_val*2 + c_val)
              print('Rates:', cc, int((c_train.sum())/c_train.shape[0]*100), rr, int((y_train[c_train==0].sum())/y_train[c_train==0].shape[0]*100))
              print('n:', n[i], 'train:', int(n[i]*(1-test_size)*(1-test_size)), 'val:', int(n[i]*(1-test_size)*test_size), 
                    'y val sum:', y_val[c_val==0].sum(), 'c val sum:', c_val.sum(), ' y1c1 val sum:', y_val[c_val==1].sum())
              
              data_dict['X_train'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = X_train
              data_dict['y_train'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = y_train
              data_dict['c_train'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = c_train
              data_dict['X_val'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = X_val
              data_dict['y_val'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = y_val
              data_dict['c_val'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = c_val
              # test dataset will be same for different n[i]s of train/val data
              # but naming convention is different just for programming
              data_dict['X_test'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = data_dict['X_test'+str(n[0])+'R'+str(rr)+'C'+str(cc)]
              data_dict['y_test'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = data_dict['y_test'+str(n[0])+'R'+str(rr)+'C'+str(cc)]
              data_dict['c_test'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = data_dict['c_test'+str(n[0])+'R'+str(rr)+'C'+str(cc)]

  return data_dict

### Generate data for visualisation

In [ ]:
# Generate synthetic classification data with 2 features and 100 data points
np.random.seed(42)  # Set seed for reproducibility
num_samples = 1500
num_features = 2

m = np.random.uniform(-1, 1, size=num_features)  # Slope/coefficients for features
c = np.random.uniform(-5, 5)  # Intercept

data, labels, sensoring = generate_classification_data(m, c, num_samples, num_features, nonlinear=True, nlband=10)
# filter data as per required risk and censoring
# data, labels, sensoring = filter_data(data, labels, sensoring, 2000, .40, .40)

print('labels.shape:', labels.shape, 'labels.sum():', labels[sensoring==0].sum(), 'sensoring.sum():', sensoring.sum(), sensoring.sum()/labels.shape[0])

# # # Perform t-SNE on the data to reduce it to 2 dimensions for visualization
# # tsne = TSNE(n_components=2, random_state=42, perplexity=60)
# # data_tsne = tsne.fit_transform(data)
# # # Plot the t-SNE transformed data
# # plt.figure(figsize=(8, 6))
# # plt.scatter(data_tsne[:, 0], data_tsne[:, 1], c=labels, cmap='viridis', edgecolors='k')
# # plt.title('t-SNE Visualization of Synthetic Data with Labels')
# # plt.colorbar(label='Labels')
# # plt.grid(True)
# # plt.show()

# # # labels[(labels == 0) & (sensoring == 0)] = 0
# # # labels[(labels == 0) & (sensoring == 1)] = 1
# # # labels[(labels == 1) & (sensoring == 0)] = 0
# # # labels[(labels == 1) & (sensoring == 1)] = 1
# labels[(sensoring == 1)] = 3
# # # Plot the data points
plt.figure(figsize=(8, 6), frameon=False)
# Define a custom mapping between labels and colors
color_map = {0: 'red', 1: 'blue', 3: 'yellow'}
marker_map = {0: '+', 1: '_', 3:'*'}
label_map = {0: 'Low Risk', 1: 'High Risk', 3: 'Sensored'}
# plt.scatter(data[:, 0], data[:, 1], c=labels, edgecolors='k')#, label=color_map.keys())#, cmap='viridis')
# Add labels for each category
for label, color in color_map.items():
    # if label==3:
    #     continue
    plt.scatter(data[labels == label][:, 0], data[labels == label][:, 1], label=label_map[label], marker=marker_map[label], color='black')
# plt.scatter(data[labels==0][:, 0], data[labels==0][:, 1], label='live', edgecolors='k')
# plt.scatter(data[labels==1][:, 0], data[labels==1][:, 1], label='dead', edgecolors='k')
# plt.scatter(data[labels==3][:, 0], data[labels==3][:, 1], label='sensored', edgecolors='k')
# plt.title('Generated Data with Binary Feature and Labels')
plt.xlabel(None, fontsize=16)
plt.ylabel(None, fontsize=16)
# plt.colorbar(label='Labels')
plt.xticks([], fontsize=14)
plt.yticks([],fontsize=14)
# plt.legend(fontsize=16, loc='upper right')
# plt.grid(True)
plt.show()

### Generate synthetic data


In [ ]:
data_dict = get_dataset_combinations(risk_rate=[.4, .3, .2, .1, 0.05], censoring_rate=[.4, .3, .2, .1, 0.05], n=[5000, 4000, 3000, 2000, 1000, 700])

with open('/data/Vinod/Data/datasets/SelectioinBias/data/selection_bias_data.pkl', 'wb') as file:
    pickle.dump(data_dict, file)

## Diabetes Data Preparation

In [ ]:
# functions to sample data to study risk rate and censoring
def sample_y_c_old(X, y, c, y_r, c_r, scale_cols, test_size=0.2):
    # converting ratio to numbers
    y_r = [int(X[c==0].shape[0] * (r/(1+r))) for r in y_r]
    c_r = [int(X.shape[0] * (r/(1+r))) for r in c_r]

    # saving origional data
    X_train, X_test, y_train, y_test, c_train, c_test = train_test_split(X, y, c, test_size=test_size, random_state=0, stratify= y*2 + c)
    X_train, X_val, y_train, y_val, c_train, c_val = train_test_split(X_train, y_train, c_train, test_size=test_size, random_state=0, 
                                                                      stratify= y_train*2 + c_train)

    # save actual data
    data_dict = {}
    data_dict['X_train'+str('X')+'R'+str('X')+'C'+str('X')] = X_train
    data_dict['y_train'+str('X')+'R'+str('X')+'C'+str('X')] = y_train
    data_dict['c_train'+str('X')+'R'+str('X')+'C'+str('X')] = c_train
    data_dict['X_val'+str('X')+'R'+str('X')+'C'+str('X')] = X_val
    data_dict['y_val'+str('X')+'R'+str('X')+'C'+str('X')] = y_val
    data_dict['c_val'+str('X')+'R'+str('X')+'C'+str('X')] = c_val
    data_dict['X_test'+str('X')+'R'+str('X')+'C'+str('X')] = X_test
    data_dict['y_test'+str('X')+'R'+str('X')+'C'+str('X')] = y_test
    data_dict['c_test'+str('X')+'R'+str('X')+'C'+str('X')] = c_test
    
    # sort in descending order
    y_r = sorted(y_r, reverse=True)
    c_r = sorted(c_r, reverse=True)
    
    for c_r_i in c_r:
        for y_r_i in y_r:
            X, y, c = sample_censoring(X.copy(), y.copy(), c.copy(), c_r_i)
            X, y, c = sample_yrate(X.copy(), y.copy(), c.copy(), y_r_i)

            # split train-val-test,
            X_train, X_test, y_train, y_test, c_train, c_test = train_test_split(X, y, c, test_size=test_size, random_state=0, stratify= y*2 + c)
            X_train, X_val, y_train, y_val, c_train, c_val = train_test_split(X_train, y_train, c_train, test_size=test_size, random_state=0, 
                                                                            stratify= y_train*2 + c_train)
            
            # scale data
            # scale_cols = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth', 'Age', 'Education','Income']
            scaler = RobustScaler().fit(X_train[scale_cols])
            X_train[scale_cols] = scaler.transform(X_train[scale_cols])
            X_val[scale_cols] = scaler.transform(X_val[scale_cols])
            X_test[scale_cols] = scaler.transform(X_test[scale_cols])

            # and put in dictionary
            data_dict['X_train'+str('X')+'R'+str('y_r_i')+'C'+str('c_r_i')] = X_train
            data_dict['y_train'+str('X')+'R'+str('y_r_i')+'C'+str('c_r_i')] = y_train
            data_dict['c_train'+str('X')+'R'+str('y_r_i')+'C'+str('c_r_i')] = c_train
            data_dict['X_val'+str('X')+'R'+str('y_r_i')+'C'+str('c_r_i')] = X_val
            data_dict['y_val'+str('X')+'R'+str('y_r_i')+'C'+str('c_r_i')] = y_val
            data_dict['c_val'+str('X')+'R'+str('y_r_i')+'C'+str('c_r_i')] = c_val
            data_dict['X_test'+str('X')+'R'+str('y_r_i')+'C'+str('c_r_i')] = X_test
            data_dict['y_test'+str('X')+'R'+str('y_r_i')+'C'+str('c_r_i')] = y_test
            data_dict['c_test'+str('X')+'R'+str('y_r_i')+'C'+str('c_r_i')] = c_test
    
    return data_dict

def create_bias(X, seed, threshold, bins=100, plot=False):
    np.random.seed(seed)
    w = 2 * np.random.rand(X.shape[1]) - 1
    temp = subtract_mean_from_each_column(X.copy())
    temp = (temp * w[None,:]).sum(axis=1)
    temp_std = temp.std(axis=0)

    temp = 4 * temp / temp_std
    temp = 1 / (1 + np.exp(-temp))

    c = (temp>threshold).astype(int)

    if plot:
        plt.hist(temp, bins=bins)
        plt.xlabel("Value")
        plt.ylabel("Frequency")
        plt.show()

    return c

def subtract_mean_from_each_column(df):
    for col in df.columns:
        df[col] -= df[col].mean()
    return df

def sample_censoring(X, y, c, n):
    # separate censored units
    X_c = X[c==1]
    y_c = y[c==1]
    indices = np.arange(len(X_c))
    np.random.shuffle(indices)
    X_c = X_c.iloc[indices[:n]]
    y_c = y_c.iloc[indices[:n]]

    X = pd.concat([X[c!=1], X_c], ignore_index=True)
    y = pd.concat([y[c!=1], y_c], ignore_index=True)
    c = c[:X.shape[0]]
    c.iloc[:-X_c.shape[0]] = 0
    c.iloc[-X_c.shape[0]:] = 1
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X.iloc[indices].reset_index(drop=True)
    y = y.iloc[indices].reset_index(drop=True)
    c = c.iloc[indices].reset_index(drop=True)

    return X, y, c

def sample_yrate(X, y, c, n):
    X_y = X[(y==1) & (c==0)]
    c_y = c[(y==1) & (c==0)]
    indices = np.arange(len(X_y))
    np.random.shuffle(indices)
    X_y = X_y.iloc[indices[:n]]
    c_y = c_y.iloc[indices[:n]]

    X = pd.concat([X[~((y==1) & (c==0))], X_y], ignore_index=True)
    c = pd.concat([c[~((y==1) & (c==0))], c_y], ignore_index=True)
    y = y[:X.shape[0]]
    y[:-X_y.shape[0]] = 0
    y[-X_y.shape[0]:] = 1
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X.iloc[indices].reset_index(drop=True)
    y = y.iloc[indices].reset_index(drop=True)
    c = c.iloc[indices].reset_index(drop=True)

    return X, y, c

def sample_datasets(X, y, c, y_r, c_r, n, scale_cols, test_size=0.2):
    '''
    This function will create datasets with different combinations of rates and sizes.
        X - dataframe, containing features
        y - labels
        c - censoring indicator
        n - is total number of data points
        c_r - rate of censoring and calculated on n
        y_r - event rate and calculated on uncensored data (i.e., after removing censored units)
        scale_cols - list of features to be scaled
        test_size - test and validation split fraction

    '''
    # saving origional data
    X_train, X_test, y_train, y_test, c_train, c_test = train_test_split(X.copy(), y.copy(), c.copy(), test_size=test_size, random_state=0, stratify= y*2 + c)
    X_train, X_val, y_train, y_val, c_train, c_val = train_test_split(X_train, y_train, c_train, test_size=test_size, random_state=0, 
                                                                      stratify= y_train*2 + c_train)
    # scale data
    # print(X_train[scale_cols])
    scaler = RobustScaler().fit(X_train[scale_cols])
    X_train[scale_cols] = scaler.transform(X_train[scale_cols])
    X_val[scale_cols] = scaler.transform(X_val[scale_cols])
    X_test[scale_cols] = scaler.transform(X_test[scale_cols])

    # save actual data
    data_dict = {}
    data_dict['X_train'+str('X')+'R'+str('X')+'C'+str('X')] = X_train.values
    data_dict['y_train'+str('X')+'R'+str('X')+'C'+str('X')] = y_train.values
    data_dict['c_train'+str('X')+'R'+str('X')+'C'+str('X')] = c_train.values
    data_dict['X_val'+str('X')+'R'+str('X')+'C'+str('X')] = X_val.values
    data_dict['y_val'+str('X')+'R'+str('X')+'C'+str('X')] = y_val.values
    data_dict['c_val'+str('X')+'R'+str('X')+'C'+str('X')] = c_val.values
    data_dict['X_test'+str('X')+'R'+str('X')+'C'+str('X')] = X_test.values
    data_dict['y_test'+str('X')+'R'+str('X')+'C'+str('X')] = y_test.values
    data_dict['c_test'+str('X')+'R'+str('X')+'C'+str('X')] = c_test.values
    
    # sort in descending order
    y_r = sorted(y_r, reverse=True)
    c_r = sorted(c_r, reverse=True)
    n = sorted(n, reverse=True)
    
    for rr in y_r:
      for cc in c_r:
          print('------------------------------------')
          X1, y1, c1 = filter_data(X.copy(), y.copy(), c.copy(), n=n[0], p_y=rr, p_c=cc, dataframe=True)
        #   print('Rates:', cc, int((c1.sum())/n[0]*100), rr, int((y1[c1==0].sum())/y1[c1==0].shape[0]*100))

          X_train, X_test, y_train, y_test, c_train, c_test = train_test_split(X1, y1, c1, test_size=test_size, random_state=0, stratify= y1*2 + c1)
          X_train, X_val, y_train, y_val, c_train, c_val = train_test_split(X_train, y_train, c_train, test_size=test_size, random_state=0, 
                                                                            stratify= y_train*2 + c_train)
        #   print('Rates:', cc, int((c_train.sum())/c_train.shape[0]*100), rr, int((y_train[c_train==0].sum())/y_train[c_train==0].shape[0]*100))
        #   print('Rates:', n[0], c_train.shape[0], c_val.shape[0], cc, int((c_val.sum())/c_val.shape[0]*100), int((c_val.sum())), rr, int((y_val[c_val==0].sum())/y_val[c_val==0].shape[0]*100),  int((y_train[c_train==0].sum())))
          # scale data
          scaler = RobustScaler().fit(X_train[scale_cols])
          X_train[scale_cols] = scaler.transform(X_train[scale_cols])
          X_val[scale_cols] = scaler.transform(X_val[scale_cols])
          X_test[scale_cols] = scaler.transform(X_test[scale_cols])
          
          data_dict['X_train'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = X_train.values
          data_dict['y_train'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = y_train.values
          data_dict['c_train'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = c_train.values
          data_dict['X_val'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = X_val.values
          data_dict['y_val'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = y_val.values
          data_dict['c_val'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = c_val.values
          data_dict['X_test'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = X_test.values
          data_dict['y_test'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = y_test.values
          data_dict['c_test'+str(n[0])+'R'+str(rr)+'C'+str(cc)] = c_test.values
          
          for i in range(1, len(n)):
              # we know total size of dataset whihc has to be split into train/val/test. Further
              # desired dataset to be subset of bigger dataset
              # check least populated and remove if there is only one sample to avoid splitting error.
              X_train, y_train, c_train = check_least_populated(X_train, y_train, c_train)
              X_val, y_val, c_val = check_least_populated(X_val, y_val, c_val)
              
            #   print(X_train.shape, n[i-1], int(n[i-1]*(1-test_size)*(1-test_size)), int(n[i]*(1-test_size)*(1-test_size)))
              _, X_train, _, y_train, _, c_train = train_test_split(X_train, y_train, c_train, test_size=int(n[i]*(1-test_size)*(1-test_size)), 
                                                                    random_state=42, stratify=y_train*2 + c_train)
              _, X_val, _, y_val, _, c_val = train_test_split(X_val, y_val, c_val, test_size=int(n[i]*(1-test_size)*test_size), 
                                                              random_state=42, stratify=y_val*2 + c_val)
              print('Rates:', cc, int((c_train.sum())/c_train.shape[0]*100), rr, int((y_train[c_train==0].sum())/y_train[c_train==0].shape[0]*100))
              print('n:', n[i], 'train:', int(n[i]*(1-test_size)*(1-test_size)), 'val:', int(n[i]*(1-test_size)*test_size), 
                    'y val sum:', y_val[c_val==0].sum(), 'c val sum:', c_val.sum(), ' y1c1 val sum:', y_val[c_val==1].sum())
              
              data_dict['X_train'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = X_train.values
              data_dict['y_train'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = y_train.values
              data_dict['c_train'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = c_train.values
              data_dict['X_val'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = X_val.values
              data_dict['y_val'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = y_val.values
              data_dict['c_val'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = c_val.values
              # test dataset will be same for different n[i]s of train/val data
              # but naming convention is different just for programming
              data_dict['X_test'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = data_dict['X_test'+str(n[0])+'R'+str(rr)+'C'+str(cc)]
              data_dict['y_test'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = data_dict['y_test'+str(n[0])+'R'+str(rr)+'C'+str(cc)]
              data_dict['c_test'+str(n[i])+'R'+str(rr)+'C'+str(cc)] = data_dict['c_test'+str(n[0])+'R'+str(rr)+'C'+str(cc)]

    return data_dict


In [ ]:
# https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset/data
with open('paths.json', 'r') as json_file:
    file_path = json.load(json_file)['diabetes_binary_5050split_health_indicators_BRFSS2015.csv']

df = pd.read_csv(file_path)
y = df['Diabetes_binary']
X = df.drop(['Diabetes_binary'], axis=1)
c = create_bias(X, seed=44, threshold=0.8)
print(X.shape, y.shape, y[c==0].sum()/y[c==0].shape[0], c.sum()/c.shape[0])

scale_cols = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth', 'Age', 'Education','Income']
data_dict = sample_datasets(X, y, c, y_r=[.4, .3, .2, .1, 0.05], c_r=[.4, .3, .2, .1, 0.05], n=[25000, 10000, 5000, 2000, 1000, 700], scale_cols=scale_cols, test_size=0.2)

# # check dataset sizes
# print(data_dict['X_train700R0.1C0.1'].shape[0]+data_dict['X_val700R0.1C0.1'].shape[0]+data_dict['X_test700R0.1C0.1'].shape[0])
# print(data_dict['c_train700R0.1C0.1'].sum()+data_dict['c_val700R0.1C0.1'].sum()+data_dict['c_test700R0.1C0.1'].sum())
# print(data_dict['y_train700R0.1C0.1'].sum()+data_dict['y_val700R0.1C0.1'].sum()+data_dict['y_test700R0.1C0.1'].sum())
# print(data_dict['y_train700R0.1C0.1'][data_dict['c_train700R0.1C0.1']==0].sum()+data_dict['y_val700R0.1C0.1'][data_dict['c_val700R0.1C0.1']==0].sum()+data_dict['y_test700R0.1C0.1'][data_dict['c_test700R0.1C0.1']==0].sum())

# save
with open('paths.json', 'r') as json_file:
    file_path = json.load(json_file)['diabetes_bias_data.pkl']
with open(file_path, 'wb') as file:
    pickle.dump(data_dict, file)

## Covid-19 Data

In [ ]:
def unique_values_per_column(df):
  """
  This function finds the unique values for each column in a pandas DataFrame.

  Args:
      df: A pandas DataFrame.

  Returns:
      A dictionary where the keys are the column names and the values are lists of unique values for each column.
  """
  unique_values = {}
  for col in df.columns:
    unique_values[col] = df[col].unique()
    print(col, df[col].unique())
  return unique_values

In [ ]:
# https://www.kaggle.com/datasets/tanmoyx/covid19-patient-precondition-dataset/data
with open('paths.json', 'r') as json_file:
    file_path = json.load(json_file)['covid_processed.csv']

df = pd.read_csv(file_path)

# result = unique_values_per_column(df.copy())
# print(result)

# changing range to 0-1 from 1-2
df['intubed'] = df['intubed'] -1
df['pneumonia'] = df['pneumonia'] -1
df['pregnancy'] = df['pregnancy'] -1
df['diabetes'] = df['diabetes'] -1
df['copd'] = df['copd'] -1
df['asthma'] = df['asthma'] -1
df['inmsupr'] = df['inmsupr'] -1
df['hypertension'] = df['hypertension'] -1
df['other_disease'] = df['other_disease'] -1
df['cardiovascular'] = df['cardiovascular'] -1
df['obesity'] = df['obesity'] -1
df['renal_chronic'] = df['renal_chronic'] -1
df['tobacco'] = df['tobacco'] -1
df['contact_other_covid'] = df['contact_other_covid'] -1
df['icu'] = df['icu'] -1

# result = unique_values_per_column(df.copy())
# print(result)

# since +ve class is more so inverting y
y = 1 - df['Chance']
X = df.drop(['Chance'], axis=1)
# creating bias in the data
c = create_bias(X, seed=42, threshold=0.895)
print(X.shape, y.shape, y[c==0].sum()/y[c==0].shape[0], c.sum()/c.shape[0])

# sample and create datasets
scale_cols = ['age']
data_dict = sample_datasets(X, y, c, y_r=[.4, .3, .2, .1, 0.05], c_r=[.4, .3, .2, .1, 0.05], n=[15000, 10000, 5000, 2000, 1000, 700], scale_cols=scale_cols, test_size=0.2)

# save data
with open('paths.json', 'r') as json_file:
    file_path = json.load(json_file)['covid_bias_data.pkl']
with open(file_path, 'wb') as file:
    pickle.dump(data_dict, file)
